# Connected components

-----

## Setup

Here are loaded the packages used in the rest of notebook:

In [ ]:
use Graph;
use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Wheel;
use Graph::Path;
use Graph::Petersen;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Math::DistanceFunctions;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [ ]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

------

## Highlight connected components

Make a grid graph with randomly directed edges:

In [ ]:
my $g = Graph::Grid.new(10, 20, :!directed);
my $g2 = $g.directed-graph(method => 'random');

Find the connected components in the graph:

In [ ]:
my @components = $g2.connected-components.grep(*.elems - 1);
@components».elems;

Highlight the connected components in a graph:

In [ ]:
#% js
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_').reverse».Int });

$g2.edges(:dataset) ==> 
js-d3-graph-plot(
    :%vertex-coordinates,
    highlight => @components,
    directed => $g2.directed,
    :$background, 
    :$title-color, 
    width => 1200, 
    height => 600, 
    vertex-label-color => 'none',
    :$edge-thickness,
    vertex-size => 12, 
    force => {charge => {strength => -200, iterations => 2}, collision => {radius => 30, iterations => 6}, link => {minDistance => 1}}
)

--------

## Lily pond frog jumping

A frog in a lily pond is able to jump 0.75 meters to get from one of the 25 lily pads to another. Model the frog's jumping network from the lily leaf density and [`SpatialGraphDistribution`](http://reference.wolfram.com/language/ref/SpatialGraphDistribution.html):

In [ ]:
# Assumming the pond is 10x10 meters.
my $g = Graph::Random.new(Graph::Distribution::Spatial.new(30, 0.75 / 10))

In [ ]:
my @points;
@points.append([random-variate(NormalDistribution.new(0, 1)), random-variate(NormalDistribution.new(0, 1))] xx 10);
@points.append([random-variate(NormalDistribution.new(-1, 1)), random-variate(NormalDistribution.new(4, 1))] xx 10);
@points.append([random-variate(NormalDistribution.new(4, 1)), random-variate(NormalDistribution.new(4, 1))] xx 10);

@points.elems;

In [ ]:
my $g = Graph::Random.new(Graph::Distribution::Spatial.new(@points.elems, 0));
$g.vertex-coordinates = (@points.kv).Hash;
for ($g.vertex-coordinates.keys X $g.vertex-coordinates.keys) -> $p {
    if $p.head ne $p.tail && euclidean-distance($g.vertex-coordinates{$p.head}, $g.vertex-coordinates{$p.tail}) < 1.2 {
        $g.add-edge($p.head, $p.tail)
    }
}

In [ ]:
#% js
$g.edges(:dataset) ==> 
js-d3-graph-plot(
    highlight => $g.connected-components,
    vertex-coordinates => $g.vertex-coordinates,
    directed => $g.directed,
    background => 'Teal', 
    :$title-color, 
    width => 600, 
    height => 600, 
    edge-thickness => 0,
    edge-color => 'Silver',
    vertex-size => 8,
    vertex-color => 'Green',
    vertex-label-color => 'none',
    force => {charge => {strength => -200, iterations => 2}, collision => {radius => 30, iterations => 6}, link => {minDistance => 1}}
)

------

## Topological sorting (via Tarjan's algorithm)

Tarjan's strongly connected components algorithm can be used to find topological sorting. 

Here is a graph from Wikipedia's article ["Topological sorting"](https://en.wikipedia.org/wiki/Topological_sorting):

In [ ]:
my @edges = '1_1' => '2_1', '1_2' => '2_1', '2_1' => '3_1', '2_1' => '3_2', '2_1' => '3_3', '1_2' => '2_2', '1_3' => '2_2', '1_3' => '2_2', '2_2' => '3_2';
@edges .= map({ $_.key.Str => $_.value.Str }); 
my $g = Graph.new(@edges, :directed);

my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_').reverse».Int });


In [ ]:
#% js
$g.edges(:dataset) ==> 
js-d3-graph-plot(
    #:%vertex-coordinates,
    directed => $g.directed,
    :$background, 
    :$title-color, 
    width => 600, 
    height => 400, 
    :$edge-thickness,
    vertex-size => 12, 
    force => {charge => {strength => -200, iterations => 2}, collision => {radius => 30, iterations => 6}, link => {minDistance => 1}}
)

In [ ]:
$g.topological-sort